# Tables for thesis

In [2]:
import pandas as pd
import numpy as np
import glob
import os
import json
import re
from pathlib import Path
from sklearn.preprocessing import StandardScaler

In [3]:
# directory management
wdir = os.getcwd()
udir = os.path.join(os.path.split(wdir)[0], "utils")
ddir = os.path.join(os.path.split(os.path.split(wdir)[0])[0], "fair_ml_thesis_data", "rawdata")
cdir = os.path.join(os.path.split(os.path.split(wdir)[0])[0], "fair_ml_thesis_data", "clustering")
adir = os.path.join(os.path.split(os.path.split(wdir)[0])[0], "fair_ml_thesis_data", "aggregated")

json_file_path = os.path.join(udir,'tasks_metadata.json')
with open(json_file_path, 'r') as j:
    task_infos = json.loads(j.read())

## Create year table with this info for each state:
- sample size
- share of positive class
- share of male vs female
- share of white/black/other
- share of privileged group in positive class (sex)
- share of privileged group in positive class (race)

In [4]:
# get all files for year y
year=2014
task='ACSEmployment'
data_file_paths = glob.glob(os.path.join(ddir, str(year), '1-Year') + f'/{str(year)}*_{task}.csv')
data_file_paths

['C:\\Users\\sarab\\Desktop\\Data_Science_MSc\\master_thesis\\fair_ml_thesis_data\\rawdata\\2014\\1-Year\\2014_AK_ACSEmployment.csv',
 'C:\\Users\\sarab\\Desktop\\Data_Science_MSc\\master_thesis\\fair_ml_thesis_data\\rawdata\\2014\\1-Year\\2014_AL_ACSEmployment.csv',
 'C:\\Users\\sarab\\Desktop\\Data_Science_MSc\\master_thesis\\fair_ml_thesis_data\\rawdata\\2014\\1-Year\\2014_AR_ACSEmployment.csv',
 'C:\\Users\\sarab\\Desktop\\Data_Science_MSc\\master_thesis\\fair_ml_thesis_data\\rawdata\\2014\\1-Year\\2014_AZ_ACSEmployment.csv',
 'C:\\Users\\sarab\\Desktop\\Data_Science_MSc\\master_thesis\\fair_ml_thesis_data\\rawdata\\2014\\1-Year\\2014_CA_ACSEmployment.csv',
 'C:\\Users\\sarab\\Desktop\\Data_Science_MSc\\master_thesis\\fair_ml_thesis_data\\rawdata\\2014\\1-Year\\2014_CO_ACSEmployment.csv',
 'C:\\Users\\sarab\\Desktop\\Data_Science_MSc\\master_thesis\\fair_ml_thesis_data\\rawdata\\2014\\1-Year\\2014_CT_ACSEmployment.csv',
 'C:\\Users\\sarab\\Desktop\\Data_Science_MSc\\master_thesis\\

In [5]:
state_pattern = re.compile(r"_([^_]+)_")
test_path = Path(data_file_paths[0])
print(state_pattern.findall(test_path.name)[-1])

AK


In [10]:
table_1 = {'state code':[],'sample size (cleaned)':[], 'employed (rate, %)':[],
           'male (rate, %)':[],'female (rate, %)':[],'white (rate, %)':[],'black (rate, %)':[],'other (rate, %)':[],
           '[male,employed] (rate, %)':[], '[white,employed] (rate, %)':[]}

for p in data_file_paths:

    # load data and change RAC1P values
    df = pd.read_csv(p)
    for c in task_infos['tasks'][1]['columns']:
        df.loc[:,c] = df.loc[:,c].astype(int)

    # age between 16 and 90
    df = df[df['AGEP'].between(16,90)]
    df['RAC1P'] = pd.to_numeric(df['RAC1P'])
    df.loc[df['RAC1P'] > 2, 'RAC1P'] = 3

    # state name
    table_1['state code'].append(state_pattern.findall(p)[-1])
    # sample size
    table_1['sample size (cleaned)'].append(len(df))

    # percentage of employed (esr 1) in dataset
    table_1['employed (rate, %)'].append(round(df['ESR'].value_counts(normalize=True).to_dict()[1],4)*100)
    # share of male and female
    table_1['male (rate, %)'].append(round(df['SEX'].value_counts(normalize=True).to_dict()[1],4)*100)
    table_1['female (rate, %)'].append(round(df['SEX'].value_counts(normalize=True).to_dict()[2],4)*100)
    # share of white,black,other (RAC1P)
    table_1['white (rate, %)'].append(round(df['RAC1P'].value_counts(normalize=True).to_dict()[1],4)*100)
    table_1['black (rate, %)'].append(round(df['RAC1P'].value_counts(normalize=True).to_dict()[2],4)*100)
    table_1['other (rate, %)'].append(round(df['RAC1P'].value_counts(normalize=True).to_dict()[3],4)*100)
    # share of male employed                                        ,2))
    table_1['[male,employed] (rate, %)'].append(round( (len(df[ (df['SEX']==1) & (df['ESR']==1) ])/len
    (df[df['ESR']==1])), 4) *100)
    # share of white employed
    table_1['[white,employed] (rate, %)'].append(round( (len(df[ (df['RAC1P']==1) & (df['ESR']==1) ])/len
    (df[df['ESR']==1])), 4) *100)
table1_final = pd.DataFrame.from_dict(table_1)
table1_final

C:\Users\sarab\AppData\Local\Temp\ipykernel_1688\1843594873.py:10: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:,c] = df.loc[:,c].astype(int)
C:\Users\sarab\AppData\Local\Temp\ipykernel_1688\1843594873.py:10: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:,c] = df.loc[:,c].astype(int)
C:\Users\sarab\AppData\Local\Temp\ipykernel_1688\1843594873.py:10: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.c

5074
{1: 0.550453291288924, 0: 0.44954670871107605}
AK -> 53.41926244181884
39150
{0: 0.5155300127713921, 1: 0.4844699872286079}
AL -> 51.94284810460273
23861
{0: 0.5115460374669963, 1: 0.48845396253300366}
AR -> 52.62977262977263


C:\Users\sarab\AppData\Local\Temp\ipykernel_1688\1843594873.py:10: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:,c] = df.loc[:,c].astype(int)


52589
{1: 0.5032611382608531, 0: 0.496738861739147}
AZ -> 52.796040202524


C:\Users\sarab\AppData\Local\Temp\ipykernel_1688\1843594873.py:10: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:,c] = df.loc[:,c].astype(int)


...

199010
{1: 0.5622230038691522, 0: 0.4377769961308477}
TX -> 53.82436007436008
20823
{1: 0.6112952024204005, 0: 0.38870479757959947}
UT -> 56.39091837536334


C:\Users\sarab\AppData\Local\Temp\ipykernel_1688\1843594873.py:10: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:,c] = df.loc[:,c].astype(int)
C:\Users\sarab\AppData\Local\Temp\ipykernel_1688\1843594873.py:10: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:,c] = df.loc[:,c].astype(int)
C:\Users\sarab\AppData\Local\Temp\ipykernel_1688\1843594873.py:10: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.c

67316
{1: 0.5711717867966011, 0: 0.4288282132033989}
VA -> 50.95321074670343
5305
{1: 0.5924599434495759, 0: 0.40754005655042413}
VT -> 51.065860642698055
56576
{1: 0.550233314479638, 0: 0.449766685520362}
WA -> 52.540957275939604
47489
{1: 0.5943481648381731, 0: 0.40565183516182696}
WI -> 51.78742249778565
15111
{0: 0.5325921514128781, 1: 0.467407848587122}
WV -> 52.44230496955967
4574
{1: 0.5972890249234806, 0: 0.4027109750765195}
WY -> 53.294289897510986


C:\Users\sarab\AppData\Local\Temp\ipykernel_1688\1843594873.py:10: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:,c] = df.loc[:,c].astype(int)
C:\Users\sarab\AppData\Local\Temp\ipykernel_1688\1843594873.py:10: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:,c] = df.loc[:,c].astype(int)
C:\Users\sarab\AppData\Local\Temp\ipykernel_1688\1843594873.py:10: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.c

,state code,sample size (cleaned),"employed (rate, %)","male (rate, %)","female (rate, %)","white (rate, %)","black (rate, %)","other (rate, %)","[male,employed] (rate, %)","[white,employed] (rate, %)"
0,AK,5074,55.05,52.98,47.02,56.96,1.50,41.55,53.42,62.51
1,AL,39150,48.45,47.29,52.71,73.20,23.44,3.37,51.94,75.53
2,AR,23861,48.85,48.03,51.97,82.30,13.56,4.14,52.63,83.46
3,AZ,52589,50.33,49.35,50.65,79.12,3.52,17.36,52.80,80.08
4,CA,298838,53.70,49.07,50.93,63.79,5.34,30.86,53.66,63.49
5,CO,42090,60.10,49.59,50.41,87.82,3.15,9.03,52.88,88.44
6,CT,29475,58.21,47.82,52.18,82.03,8.44,9.54,50.92,82.88
7,DE,7406,54.46,48.19,51.81,76.79,16.50,6.71,50.38,76.64
8,FL,160571,50.11,47.70,52.30,80.36,13.30,6.34,50.91,80.00
9,GA,77261,52.75,47.94,52.06,66.80,26.55,6.65,51.52,68.42


In [29]:
#table1_final.style.to_latex()
print(table1_final.to_latex())

\begin{tabular}{llrrrrrrrrr}
\toprule
{} & state code &  sample size (cleaned) &  employed (rate, \%) &  male (rate, \%) &  female (rate, \%) &  white (rate, \%) &  black (rate, \%) &  other (rate, \%) &  [male,employed] (rate, \%) &  [white,employed] (rate, \%) \\
\midrule
0  &         AK &                   5155 &               54.06 &           52.86 &             47.14 &            59.13 &             1.84 &            39.03 &                      53.14 &                       62.33 \\
1  &         AL &                  39366 &               49.71 &           47.64 &             52.36 &            75.16 &            21.08 &             3.76 &                      52.38 &                       76.98 \\
2  &         AR &                  24454 &               50.27 &           48.13 &             51.87 &            82.93 &            11.65 &             5.43 &                      51.90 &                       83.75 \\
3  &         AZ &                  56420 &               51.70 & 

C:\Users\sarab\AppData\Local\Temp\ipykernel_15236\792255623.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(table1_final.to_latex())


## Table with domain knowledge groups for each year

In [3]:
json_file_path = os.path.join(udir,'clustering_infos.json')
with open(json_file_path, 'r') as j:
    clustering_infos = json.loads(j.read())

In [4]:
clustering_infos

{'BEA_clustering': {'2014': {'0': ['ME',
    'VT',
    'RI',
    'DE',
    'ND',
    'SD',
    'MT',
    'ID',
    'WY',
    'AK'],
   '1': ['NH', 'WV', 'MS', 'AR', 'KS', 'NE', 'NM', 'UT', 'NV', 'HI'],
   '2': ['CT', 'KY', 'SC', 'AL', 'IA', 'MO', 'LA', 'OK', 'AZ', 'OR'],
   '3': ['MA', 'MD', 'VA', 'TN', 'IN', 'MI', 'WI', 'MN', 'CO', 'WA'],
   '4': ['CA', 'TX', 'IL', 'FL', 'GA', 'NC', 'OH', 'PA', 'NY', 'NJ']},
  '2015': {'0': ['ME', 'VT', 'RI', 'WV', 'ND', 'SD', 'MT', 'ID', 'WY', 'AK'],
   '1': ['NH', 'DE', 'MS', 'AR', 'KS', 'NE', 'NM', 'UT', 'NV', 'HI'],
   '2': ['CT', 'KY', 'SC', 'AL', 'IA', 'MO', 'LA', 'OK', 'AZ', 'OR'],
   '3': ['MA', 'MD', 'VA', 'TN', 'IN', 'MI', 'WI', 'MN', 'CO', 'WA'],
   '4': ['CA', 'TX', 'IL', 'FL', 'GA', 'NC', 'OH', 'PA', 'NY', 'NJ']},
  '2016': {'0': ['ME', 'VT', 'RI', 'DE', 'ND', 'SD', 'MT', 'ID', 'WY', 'AK'],
   '1': ['NH', 'WV', 'MS', 'AR', 'KS', 'NE', 'NM', 'UT', 'NV', 'HI'],
   '2': ['CT', 'KY', 'SC', 'AL', 'IA', 'MO', 'LA', 'OK', 'AZ', 'OR'],
   '3': ['

In [5]:
prova = pd.DataFrame.from_dict(clustering_infos['BEA_clustering'], orient='index')
groups = prova.T

In [6]:
groups= groups.rename(index={'0': 'Group 1','1':'Group 2','2':'Group 3','3':'Group 4','4':'Group 5'})

In [7]:
groups

,2014,2015,2016,2017,2018
Group 1,"[ME, VT, RI, DE, ND, SD, MT, ID, WY, AK]","[ME, VT, RI, WV, ND, SD, MT, ID, WY, AK]","[ME, VT, RI, DE, ND, SD, MT, ID, WY, AK]","[ME, VT, RI, DE, ND, SD, MT, ID, WY, AK]","[ME, VT, RI, DE, ND, SD, MT, ID, WY, AK]"
Group 2,"[NH, WV, MS, AR, KS, NE, NM, UT, NV, HI]","[NH, DE, MS, AR, KS, NE, NM, UT, NV, HI]","[NH, WV, MS, AR, KS, NE, NM, UT, NV, HI]","[NH, WV, MS, AR, KS, NE, NM, UT, NV, HI]","[NH, WV, MS, AR, KS, NE, NM, UT, NV, HI]"
Group 3,"[CT, KY, SC, AL, IA, MO, LA, OK, AZ, OR]","[CT, KY, SC, AL, IA, MO, LA, OK, AZ, OR]","[CT, KY, SC, AL, IA, MO, LA, OK, AZ, OR]","[CT, KY, SC, AL, IA, MO, LA, OK, OR, WI]","[CT, KY, SC, AL, IA, MO, LA, OK, OR, WI]"
Group 4,"[MA, MD, VA, TN, IN, MI, WI, MN, CO, WA]","[MA, MD, VA, TN, IN, MI, WI, MN, CO, WA]","[MA, MD, VA, TN, IN, MI, WI, MN, CO, WA]","[AZ, MA, MD, VA, TN, IN, MI, MN, CO, WA]","[AZ, MA, MD, VA, TN, IN, MI, MN, CO, WA]"
Group 5,"[CA, TX, IL, FL, GA, NC, OH, PA, NY, NJ]","[CA, TX, IL, FL, GA, NC, OH, PA, NY, NJ]","[CA, TX, IL, FL, GA, NC, OH, PA, NY, NJ]","[CA, TX, IL, FL, GA, NC, OH, PA, NY, NJ]","[CA, TX, IL, FL, GA, NC, OH, PA, NY, NJ]"


In [12]:
print(groups['2018'].to_latex(caption='Groups of US states following US Bureau of Economic Analysis GDP reports'))

\begin{table}
\centering
\caption{Groups of US states following US Bureau of Economic Analysis GDP reports}
\begin{tabular}{ll}
\toprule
{} &                                      2018 \\
\midrule
Group 1 &  [ME, VT, RI, DE, ND, SD, MT, ID, WY, AK] \\
Group 2 &  [NH, WV, MS, AR, KS, NE, NM, UT, NV, HI] \\
Group 3 &  [CT, KY, SC, AL, IA, MO, LA, OK, OR, WI] \\
Group 4 &  [AZ, MA, MD, VA, TN, IN, MI, MN, CO, WA] \\
Group 5 &  [CA, TX, IL, FL, GA, NC, OH, PA, NY, NJ] \\
\bottomrule
\end{tabular}
\end{table}



C:\Users\sarab\AppData\Local\Temp\ipykernel_16256\357604081.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(groups['2018'].to_latex(caption='Groups of US states following US Bureau of Economic Analysis GDP reports'))
